In [ ]:
from supabase import create_client, Client
import os
import httpx
import json
from uuid import uuid4
from IPython.display import Markdown

SUPABASE_URL = "serverless_url"
SUPABASE_ANON_KEY = "supabase_url"

supabase: Client = create_client(SUPABASE_URL, SUPABASE_ANON_KEY)

## Calling Edge Function Endpoints with HTTPX
To call specific REST endpoints (like GET/POST on `/appointments`), it's best to use a standard HTTP client.

In [ ]:
# MUST RUN THIS CELL.

function_root_url = f"{SUPABASE_URL}/functions/v1/index"
headers = {"Authorization": f"Bearer {SUPABASE_ANON_KEY}", "Content-Type": "application/json"}

print(f"Using API root: {function_root_url}")

### 1. POST /appointments
Create a new appointment.

In [ ]:
new_appointment = {
    "appointmentid": str(uuid4()),
    "patientname": "Jane Doe",
    "doctorname": "Dr. Smith",
    "date": "2025-09-15",
    "time": "11:00:00",
    "summaryfordoctor": "Follow-up consultation for annual check-up."
}

try:
    post_response = httpx.post(f"{function_root_url}/appointments", headers=headers, json=new_appointment, timeout=10.0)
    post_response.raise_for_status() # Raises an exception for 4xx or 5xx status codes
    
    print("POST Response Status Code:", post_response.status_code)
    print("POST Response Body:")
    created_appointment = post_response.json()
    print(json.dumps(created_appointment, indent=2))
    
except httpx.HTTPStatusError as e:
    print(f"Error: {e}")
    print(f"Response body: {e.response.text}")

### 2. GET /appointments
Retrieve all appointments.

In [ ]:
try:
    get_response = httpx.get(f"{function_root_url}/appointments", headers=headers, timeout=10.0)
    get_response.raise_for_status()
    
    print("GET Response Status Code:", get_response.status_code)
    print("GET Response Body:")
    appointments_list = get_response.json()
    print(json.dumps(appointments_list, indent=2))
    
except httpx.HTTPStatusError as e:
    print(f"Error: {e}")
    print(f"Response body: {e.response.text}")

In [ ]:
import pandas as pd
try:
    get_response = httpx.get(f"{function_root_url}/appointments", headers=headers, timeout=10.0)
    get_response.raise_for_status()
    
    # Store data in variable and display as table
    df = pd.DataFrame(appointments_list)
    display(df)  # This will show the table in Jupyter
    
except httpx.HTTPStatusError as e:
    print(f"Error: {e}")
    print(f"Response body: {e.response.text}")

### 3. GET /appointments/{appointment_id}/summary
Retrieve the summary for a specific appointment and display it as markdown.

In [ ]:
# Get the raw response and save it as a variable
appointment_id = '33ee2fc7-548b-4d4a-8672-61723b8a61ed'  # Example ID

try:
    summary_response = httpx.get(f"{function_root_url}/appointments/{appointment_id}/summary", headers=headers, timeout=10.0)
    summary_response.raise_for_status()
    
    print("GET Summary Response Status Code:", summary_response.status_code)
    summary_data = summary_response.json()  # Save the raw response data
    
except httpx.HTTPStatusError as e:
    print(f"Error: {e}")
    print(f"Response body: {e.response.text}")
    summary_data = None  # Set to None on error
    
# Display the clinical summary as markdown (if the request succeeded)
if 'summary_data' in locals() and summary_data:
    summary_text = summary_data.get('summary', 'No summary available')
    display(Markdown(f"{summary_text}"))
else:
    print("No summary data available to display.")

### 4. PUT /appointments/{appointment_id}
Update an existing appointment, such as populating the `consultationsummary` field.

In [ ]:
# Update an existing appointment (e.g., set consultationsummary)
appointment_id = 'a1b2c3d4-e5f6-7890-1234-567890abcdef'  # Replace with actual appointment ID from your data

update_data = {
    "consultationsummary": "Updated consultation notes after the appointment."
}

try:
    put_response = httpx.put(f"{function_root_url}/appointments/{appointment_id}", headers=headers, json=update_data, timeout=10.0)
    put_response.raise_for_status()
    
    print("PUT Response Status Code:", put_response.status_code)
    print("PUT Response Body:")
    updated_appointment = put_response.json()
    print(json.dumps(updated_appointment, indent=2))
    
except httpx.HTTPStatusError as e:
    print(f"Error: {e}")
    print(f"Response body: {e.response.text}")

In [ ]:
# Display the updated consultationsummary as markdown (if the update succeeded)
if 'updated_appointment' in locals() and updated_appointment:
    consultation_summary = updated_appointment.get('consultationsummary', 'No consultation summary available')
    display(Markdown(f"## Updated Consultation Summary\n\n{consultation_summary}"))
else:
    print("No updated data available to display.")

### 5. GET /appointments/{appointment_id}/consultationsummary
Retrieve the consultation summary for a specific appointment and display it as markdown.

In [ ]:
# Get the consultation summary for a specific appointment
appointment_id = '33ee2fc7-548b-4d4a-8672-61723b8a61ed'  # Replace with actual appointment ID

try:
    consultation_response = httpx.get(f"{function_root_url}/appointments/{appointment_id}/consultationsummary", headers=headers, timeout=10.0)
    consultation_response.raise_for_status()
    
    consultation_data = consultation_response.json()  # Save the response data
    
    if consultation_data.get('consultationsummary'):
        display(Markdown(f"## Consultation Summary\n\n{consultation_data['consultationsummary']}"))
    
except httpx.HTTPStatusError as e:
    print(f"Error: {e}")
    print(f"Response body: {e.response.text}")
    consultation_data = None